-------------------------------------------Myntra Data Scraping Project---------------------------------

Harvesting the Product Information :

In [1]:
#importing Necessary Libraries
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import pandas as pd
import re

In [2]:
#Loading Product links ".csv" file into pandas dataframe
product_link_df=pd.read_csv(r"myntra_t_shirts_product_link.csv")
product_link_df.head(5)

,product_link
0,https://www.myntra.com/tshirts/stormborn/storm...
1,https://www.myntra.com/tshirts/adidas/adidas-m...
2,https://www.myntra.com/tshirts/force+nxt/force...
3,https://www.myntra.com/tshirts/bullmer/bullmer...
4,https://www.myntra.com/tshirts/roadster/roadst...


In [3]:
#Checking the size
print("Shape :",product_link_df.shape)
#Checking for  duplications
print("Duplicated_rows :",product_link_df.duplicated().sum())

Shape : (35736, 1)
Duplicated_rows : 0


In [ ]:
#Creating lists for storing different product information
product_name=[]
product_code=[]
product_image_url=[]
product_description=[]
material=[]
fit_type=[]
available_sizes=[]
original_price=[]
discounted_price=[]
discount_percentage=[]
total_ratings=[]
avg_rating_for_five=[]
total_text_reviews=[]
supplier_name=[]
product_link=[]
#-----------------------------------------------------------------------------------------------------------------------------------------------
#Iterating the hyperlinks of the product in dataframe 
for index,row in tqdm(product_link_df.iloc[406:606,:].iterrows(), desc="Processing", unit="iteration"):
            #Connecting with Google Chrome Browser
            driver = webdriver.Chrome()
            #--------------------------------------------------------------------------------------------
            #Code for disabling the images in webpage to reduce the iteration time for increasing efficiency  
            chrome_options = webdriver.ChromeOptions()
            prefs = {"profile.managed_default_content_settings.images": 2}
            chrome_options.add_experimental_option("prefs", prefs)
            driver = webdriver.Chrome(options=chrome_options)
            #---------------------------------------------------------------------------------------------  
            try:
                driver.get(row.iloc[0])
                driver.minimize_window()
            except Exception as e:
                print("Exception_Occured :",e)
            #product_link
            product_link.append(row.iloc[0])
            #product_name
            try:
                product_name_element=driver.find_element(By.CSS_SELECTOR,"h1.pdp-title")
                product_name.append(product_name_element.text)
            except:
                product_name.append(None)
            #material
            try:
                material_element=driver.find_element(By.XPATH,'//div[contains(@class,"index-row") and .//div[@class="index-rowKey" and contains(text(), "Fabric")]]//div[@class="index-rowValue"]')
                material.append(material_element.text)
            except:
                material.append(None)
            #description
            try:
                product_description_element=driver.find_element(By.CSS_SELECTOR,"h1.pdp-name")
                product_description.append(product_description_element.text)
            except:
                product_description.append(None)
            #original_price
            try:
                orginal_price_element=driver.find_element(By.CSS_SELECTOR,"span.pdp-mrp>s")
                original_price.append(orginal_price_element.text)
            except:
                original_price.append(None)
            #discount_percentage:
            try:
                discount_percentage_element=driver.find_element(By.CSS_SELECTOR,"span.pdp-discount") 
                match = re.search(r'\((.*?)\)', discount_percentage_element.text)
                if match:
                  result = match.group(1)
                  discount_percentage.append( result)
            except:
                discount_percentage.append(None)
            #discounted_price
            try:
                discount_price_element=driver.find_element(By.CSS_SELECTOR,"span.pdp-price>strong")
                discounted_price.append(discount_price_element.text)
            except:
                discounted_price.append(None)
            #avg_rating_for_five
            try:
                customer_rating_element=driver.find_element(By.CSS_SELECTOR,"div.index-flexRow.index-averageRating>span")
                avg_rating_for_five.append(customer_rating_element.text)
            except:
                avg_rating_for_five.append(None)
            #product_image_url
            try:
                image_text=driver.find_element(By.CSS_SELECTOR,"div.image-grid-container.common-clearfix > div:nth-child(1) > div.image-grid-imageContainer> div").get_attribute("style")
                image_url_match = re.search(r'url\("([^"]+)"\);', image_text)
                if image_url_match:
                    image_url = image_url_match.group(1)
                    product_image_url.append(image_url)    
            except Exception as e:
                print(e)
                print("error")
                product_image_url.append(None)
            #total_ratings
            try:
                total_rating_element=driver.find_element(By.CSS_SELECTOR,"div.index-ratingsCount")
                total_ratings.append(total_rating_element.text[:-8])
            except:
                total_ratings.append(None)
            #available_sizes
            try:
                size_element=driver.find_elements(By.CSS_SELECTOR,"div.size-buttons-size-buttons>div.size-buttons-tipAndBtnContainer>div.size-buttons-buttonContainer>button>p")
                size_text_list=[i.text for i in size_element]
                available_sizes.append(",".join(map(str,size_text_list)))
            except:
                available_sizes.append(None)
            #fit_type
            try:
                fit_element=driver.find_element(By.XPATH,'//div[contains(@class,"index-row") and .//div[@class="index-rowKey" and contains(text(), "Fit")]]//div[@class="index-rowValue"]')
                fit_type.append(fit_element.text)
            except:
                fit_type.append(None)        
            #total_text_reviews
            try:
                total_reviews_element=driver.find_element(By.XPATH, '//*[@id="mountRoot"]/div/div[1]/main/div[2]/div[2]/main/div/div/div[3]')
                total_reviews_text=(total_reviews_element).text
                total_reviews=re.findall(r'\((.*?)\)',total_reviews_text)
                total_text_reviews.append(total_reviews[0])
            except:
                total_text_reviews.append(None)
            #product_code
            try:
                product_code_element=driver.find_element(By.CSS_SELECTOR,"div.supplier-supplier>span>span.supplier-styleId")
                product_code.append(product_code_element.text)
            except:
                product_code.append(None)
            #supplier_name
            try:
                supplier_element=driver.find_element(By.CSS_SELECTOR,"span.supplier-productSellerName")
                supplier_name.append(supplier_element.text)
            except:
                supplier_name.append(None)
            time.sleep(0.1)
            #finally closing the browser
            driver.quit()  
            

In [ ]:
#Creating a dictionary of product data
product_data={  
                "product_code": product_code,
                "product_name":product_name,
                "product_description": product_description,
                "material": material,
                "fit_type": fit_type,
                "available_sizes": available_sizes,
                "original_price": original_price,
                "discounted_price": discounted_price,
                "discount_percentage": discount_percentage,
                "total_ratings": total_ratings,
                "avg_rating_for_five": avg_rating_for_five,
                "total_text_reviews": total_text_reviews,
                "supplier_name": supplier_name,
                "product_image_url": product_image_url,
                "product_link":product_link
             }

In [ ]:
#Creating DataFrame of product data
product_data_df=pd.DataFrame(product_data)
product_data_df.head(5)

In [ ]:
#Checking the shape
print("Shape :",product_data_df.shape)
#Checking  duplications
print("Duplicated_rows :",product_data_df.duplicated().sum())

In [ ]:
#Converting DataFrame into ".csv" file
product_data_df.to_csv("t_shirts_data.csv",index=False,encoding="utf-8-sig")